In [ ]:
import os
import sys
import xml.etree.ElementTree as ET

In [ ]:
DOWNLOADS = './downloads'
GAME_START_ID = 1
GAME_END_ID = 2000

In [ ]:
def parseGameInfo(id, location, filename='info', allow=False):
    result = {'id':id}
    filepath = os.path.join(location, filename)
    if not os.path.isfile(filepath):
        print('File {} is missing'.format(filepath))
        if not allow:
            sys.exit(1)
        return result
    tree = ET.parse(filepath)
    root = tree.getroot()
    for child in root:
        if child.tag == 'baseImgUrl':
            result['baseImgUrl'] = child.text
        if child.tag == 'Game':
            for childChild in child:
                if childChild.tag == 'GameTitle':
                    result['GameTitle'] = childChild.text
                elif childChild.tag == 'Platform':
                    result['Platform'] = childChild.text
                elif childChild.tag == 'Overview':
                    result['Overview'] = childChild.text
                elif childChild.tag == 'Platform':
                    result['Platform'] = childChild.text
                elif childChild.tag == 'Genres':
                    result['Genres'] = []
                    for childChildChild in childChild:
                        if childChildChild.tag == 'genre':
                            result['Genres'].append(childChildChild.text)
                elif childChild.tag == 'Images':
                    result['Images'] = []
                    for childChildChild in childChild.iter(tag='screenshot'):
                        for childChildChildChild in childChildChild.iter(tag='original'):
                            result['Images'].append(childChildChildChild.text)
    return result

In [ ]:
metaData = []
totalGames = 0
totalImages = 0
for i in range(GAME_START_ID, GAME_END_ID+1):
    info = parseGameInfo(i, os.path.join(DOWNLOADS, str(i)))
    if 'Genres' in info and len(info.get('Images', [])) != 0:
        metaData.append(info)
        totalGames += 1
        totalImages += len(info['Images'])
print('Total games used: {}'.format(totalGames))
print('Total images used: {}'.format(totalImages))